In [ ]:
import pandas as pd
import json
import urllib.request as urlreq
import os

In [ ]:
path_to_data = os.path.join(os.getcwd(),'University of Missouri','DRIVERS','data')
path_to_data

In [ ]:
spark.sql("use real_world_data_ed_omop_dec_2023")

In [ ]:
cd_meta = spark.sql('''
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'DRG' and
          concept_code in (
            '765','766','767','768',
            '774','775',
            '783','784','785','786', '787','788',
            '796','797','798',
            '805','806','807'
          )
    union all 
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'CPT4' and
          concept_code in (
            '59409','59514', '59612','59620'
          )
    union all 
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'ICD10PCS' and
          concept_code in (
            '10D00Z0','10D00Z1','10D00Z2','10D07Z3','10D07Z4', '10D07Z5', '10D07Z6','10D07Z7','10D07Z8',
            '10E0XZZ'
          )
    union all
    select concept_id,concept_name,concept_code,vocabulary_id,domain_id
    from concept
    where vocabulary_id = 'LOINC' and
          concept_code in (
            
          )
''').toPandas()
cd_meta.to_csv(os.path.join(path_to_data,'cd_meta_omop.csv'),index=False)

In [ ]:
def cd_where_clause(meta_tbl,voc_id):
    '''
    generate where clause details based on omop concept_id
    '''
    cd_lst = meta_tbl.loc[meta_tbl['vocabulary_id']==voc_id,'concept_id'].tolist()
    cd_quote = []
    for code in cd_lst:
        cd_quote.append("'"+ str(code) +"'")
    cd_quote_str = ",".join(cd_quote)
    return cd_quote_str     

meta_tbl = pd.read_csv(os.path.join(path_to_data,'cd_meta_omop.csv'))
drg_where = cd_where_clause(meta_tbl,'DRG')
cpt4_where = cd_where_clause(meta_tbl,'CPT4')
icd10_where = cd_where_clause(meta_tbl,'ICD10PCS')

In [ ]:
delivery_init = spark.sql('''
    select person_id,
           visit_occurrence_id, 
           visit_detail_id, 
           observation_date as event_date, 
           observation_concept_id as event_identifier,
           'DRG' as event_source
    from observation
    where observation_concept_id in ('''+ drg_where +''')
    union all
    select person_id,
           visit_occurrence_id, 
           visit_detail_id, 
           procedure_date as event_date, 
           procedure_concept_id as event_identifier,
           'CPT4' as event_source
    from procedure_occurrence
    where procedure_concept_id in ('''+ cpt4_where +''')
    union all
    select person_id,
           visit_occurrence_id, 
           visit_detail_id, 
           procedure_date as event_date, 
           procedure_concept_id as event_identifier,
           'ICD10PCS' as event_source
    from procedure_occurrence
    where procedure_concept_id in ('''+ icd10_where +''')
''').cache()
delivery_init.createOrReplaceTempView("delivery_init")
delivery_init.head()

In [ ]:
spark.sql('''
    select * from delivery_init
    where visit_occurrence_id = '11888469770784'
    limit 5
''').toPandas()

In [ ]:
delivery_ip = spark.sql('''
    select distinct
           a.person_id, 
           a.visit_occurrence_id,
           v.visit_start_date,
           v.visit_end_date,
           v.care_site_id
    from delivery_init a 
    join visit_occurrence v 
    on a.person_id = v.person_id and 
       a.visit_occurrence_id = v.visit_occurrence_id
    where v.visit_concept_id in (
        9201, -- IP
        262   -- ER to IP
    )
''').cache()
delivery_ip.createOrReplaceTempView("delivery_ip")
delivery_ip.head()

In [ ]:
delivery_consolidate = spark.sql('''
    with cd_filter as (
        select v.person_id, 
               v.visit_occurrence_id,
               v.care_site_id,
               v.visit_start_date,
               v.visit_end_date,
               a.event_source,
               a.event_date,
               row_number() over (partition by v.person_id, v.visit_occurrence_id, a.event_source order by a.event_date) as rn_asc,
               row_number() over (partition by v.person_id, v.visit_occurrence_id, a.event_source order by a.event_date desc) as rn_desc
        from delivery_ip v
        join delivery_init a 
        on v.person_id = a.person_id and 
           v.visit_occurrence_id = a.visit_occurrence_id
        where a.event_date between date_sub(v.visit_start_date,3) and date_add(v.visit_end_date,3)
    ), f_pvt as (
        select * 
        from (
            select person_id, visit_occurrence_id,
                   event_source, event_date
            from cd_filter
            where rn_asc = 1       
        )
        pivot (
            min(event_date) for event_source in (
                'DRG' as F_DRG_DT,'CPT4' as F_CPT_DT,'ICD10PCS' as F_ICD_DT
            )
        )
    ), l_pvt as (
        select * 
        from (
            select person_id, visit_occurrence_id,
                   event_source, event_date
            from cd_filter
            where rn_desc = 1       
        )
        pivot (
            max(event_date) for event_source in (
                'DRG' as L_DRG_DT,'CPT4' as L_CPT_DT,'ICD10PCS' as L_ICD_DT
            )
        )
    )
    select a.person_id, 
           a.visit_occurrence_id,
           a.visit_start_date,
           a.visit_end_date,
           a.care_site_id,
           f.F_DRG_DT,f.F_CPT_DT,f.F_ICD_DT,
           l.L_DRG_DT,l.L_CPT_DT,l.L_ICD_DT
    from delivery_ip a 
    left join f_pvt f on a.person_id = f.person_id and a.visit_occurrence_id = f.visit_occurrence_id
    left join l_pvt l on a.person_id = l.person_id and a.visit_occurrence_id = l.visit_occurrence_id
''').cache()
delivery_consolidate.createOrReplaceTempView("delivery_consolidate")
delivery_consolidate.head()

In [ ]:
delivery_elig = spark.sql('''
    with date_consolid as (
        select distinct 
               person_id,
               visit_occurrence_id,
               care_site_id,
               coalesce(F_DRG_DT,visit_start_date,F_ICD_DT,F_CPT_DT) as event_start_dt,
               coalesce(L_DRG_DT,visit_end_date,L_ICD_DT,L_CPT_DT) as event_end_dt
        from delivery_consolidate    
    ), visit_diffs as (
        select a.*, 
               lead(a.event_start_dt, 1, '9999-12-31') OVER (PARTITION BY person_id ORDER BY event_start_dt) AS next_event_start_dt
        from date_consolid a 
    ), visit_session as (
        select b.*, 
               case 
                   when datediff(b.next_event_start_dt,b.event_start_dt) > 211 then 1
                   else 0 
               end as new_session_flag
        from visit_diffs b
    ), sessions as (
        select d.*, 
               sum(d.new_session_flag) over (PARTITION BY d.person_id ORDER BY d.event_start_dt) as event_id
        from visit_session d
    ), session_order as (
        select e.*, 
               row_number() over (partition by e.person_id, e.event_id order by e.event_start_dt) as rn,
               max(e.event_end_dt) over (partition by e.person_id, e.event_id) as event_end_date
    from sessions e
    )
    select s.person_id, 
           s.event_id, 
           s.visit_occurrence_id,
           cs.care_site_source_value,
           s.event_start_dt as event_start_date,
           s.event_end_date
    from session_order s 
    join care_site cs on s.care_site_id = cs.care_site_id
    where s.rn = 1
    order by s.person_id, s.event_id
''').cache()
delivery_elig.createOrReplaceTempView("delivery_elig")
delivery_elig.head()

In [ ]:
# delivery_elig_df = spark.createDataFrame(delivery_elig.toPandas())
# delivery_elig_df.write.parquet(os.path.join(path_to_data,'delivery_elig.parquet'),mode="overwrite")
# permission denied

In [ ]:
delivery_elig_tbl1 = spark.sql('''
    select d.person_id,
           d.event_id,
           d.event_start_date, 
           d.event_end_date,
           d.visit_occurrence_id,
           p.year_of_birth,
           p.month_of_birth,
           p.day_of_birth,
           p.race_source_value,
           p.ethnicity_source_value,
           p.location_id,
           p.care_site_id,
           d.care_site_source_value,
           tnt.bed_size,
           tnt.speciality,
           tnt.segment,
           tnt.zip_code,
           dth.death_date,
           case when dth.death_date is not null then 1 else 0 end as death_ind
    from delivery_elig d
    join person p on d.person_id = p.person_id
    left join tenant_attributes tnt on d.care_site_source_value = tnt.tenant
    left join death dth on d.person_id = dth.person_id 
''').cache()
delivery_elig_tbl1.createOrReplaceTempView("delivery_elig_tbl1")
delivery_elig_tbl1.head()

In [ ]:
import itertools
def pt_freq_qry(df,stratified_by,n_way=1):
    '''
    generate total patient counts for each stratified variables
    '''
    sql_str_lst = []
    # overall count
    sql_str_lst.append("select 'total' as summ_var,'N' as summ_cat, count(distinct person_id) as pat_cnt, count(distinct person_id || '_' || event_id) as evt_cnt from " + df)
    
    # 1-way summary
    for var_str in stratified_by:
        sql_str_lst.append(
            "select '" + var_str +"' as summ_var," 
            + "cast(" + var_str +" as string) as summ_cat,"
            + "count(distinct person_id) as pat_cnt, "
            + "count(distinct person_id || '_' || event_id) as evt_cnt "
            + "from "+ df + " group by "+ var_str
        )
        
    # up to n-way summary
    if n_way > 1:
        for L in range(2,n_way+1,1):
            for var_str_comb in itertools.combinations(stratified_by, L):
                var_str_concat_by_pipe = "|".join(var_str_comb)
                var_str_concat_by_dpipe = "|| '||' ||".join(var_str_comb)
                var_str_concat_by_comma = ",".join(var_str_comb)
                sql_str_lst.append(
                    "select 'by_" + var_str_concat_by_pipe +"' as summ_var," 
                    + "cast(" + var_str_concat_by_dpipe +" as string) as summ_cat,"
                    + "count(distinct person_id) as pat_cnt, "
                    + "count(distinct person_id || '_' || event_id) as evt_cnt "
                    + "from "+ df + " group by "+ var_str_concat_by_comma
                )
                
    # union everything
    sql_str = " union ".join(sql_str_lst)
    return(sql_str)

In [ ]:
stratified_by = [
    'race_source_value',
    'ethnicity_source_value',
    'segment',
    'speciality',
    'bed_size',
    'death_ind',
    'zip_code'
]
get_pt_summ = pt_freq_qry(
    'delivery_elig_tbl1',stratified_by,n_way=3
)
summ_stat_long = spark.sql(get_pt_summ).toPandas()
summ_stat_long.to_csv(os.path.join(path_to_data,'summ_stat.csv'),index=False)

In [ ]:
summ_stat_long = pd.read_csv(os.path.join(path_to_data,'summ_stat.csv'))